## CIFAR-10 Classifier Using CNN in PyTorch

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:

transform = transforms.Compose(
    [transforms.ToTensor(), 
# convert Python Image Library (PIL) format to PyTorch tensors.

     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
#  normalize the data by specifying a  mean and standard deviation for each of the three channels.

trainset = torchvision.datasets.CIFAR10(root='/home/CIFAR-10 Classifier Using CNN in PyTorch/data/', 
                                        train=True,
                                        download=True, 
                                        transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, 
                                          batch_size=4,
                                          shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', 
                                       train=False,
                                       download=True, 
                                       transform=transform)
testloader = torch.utils.data.DataLoader(testset, 
                                         batch_size=4,
                                         shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 
           'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
net = Net()


import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [ ]:
# import os

# model_directory_path = '/home/CIFAR-10 Classifier Using CNN in PyTorch/model/'
# model_path = model_directory_path + 'cifar-10-cnn-model.pt'

# if not os.path.exists(model_directory_path):
#     os.makedirs(model_directory_path)

# if os.path.isfile(model_path):
#     # load trained model parameters from disk
#     net.load_state_dict(torch.load(model_path))
#     print('Loaded model parameters from disk.')
# else:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 1000 == 999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 1000))
            running_loss = 0.0
print('Finished Training.')
#     torch.save(net.state_dict(), model_path)
#     print('Saved model parameters to disk.')

[1,  1000] loss: 1.981
[1,  2000] loss: 1.752
[1,  3000] loss: 1.661
[1,  4000] loss: 1.611
[1,  5000] loss: 1.570
[1,  6000] loss: 1.502
[1,  7000] loss: 1.500
[1,  8000] loss: 1.498
[1,  9000] loss: 1.489
[1, 10000] loss: 1.414


In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()
    
outputs = net(images)
images[0]

In [ ]:
total_correct = 0
total_images = 0
confusion_matrix = np.zeros([10,10], int)
# with torch.no_grad():
for data in testloader:
    images, labels = data
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)
    total_images += labels.size(0)
    total_correct += (predicted == labels).sum().item()
    for i, l in enumerate(labels):
        confusion_matrix[l.item(), predicted[i].item()] += 1 

model_accuracy = total_correct / total_images * 100
print('Model accuracy on {0} test images: {1:.2f}%'.format(total_images, model_accuracy))

